## Experiment Design

This notebook aims to compare different pivot rules for randomly generated linear programs that *always have the origin point as the starting solution*.  
To achieve this, I will consider three main categories of problems:

### **1. Balanced Problems (m = n)**  
These are square linear programs where the number of constraints equals the number of variables. I will generate instances in increments of 5, covering the following sizes:

- (5×5), (10×10), (15×15), (20×20), (25×25), (30×30), (35×35), (40×40), (45×45), (50×50)
- (55×55), (60×60), (65×65), (70×70), (75×75), (80×80), (85×85), (90×90), (95×95), (100×100)

### **2. Wide Problems (m < n, n = 2m)**  
These problems have twice as many variables as constraints, representing underdetermined systems. The chosen sizes are:

- (5×10), (10×20), (15×30), (20×40), (25×50), (30×60), (35×70), (40×80), (45×90), (50×100)

### **3. Tall Problems (m > n, m = 2n)**  
These problems have twice as many constraints as variables, representing overdetermined systems. The chosen sizes are:

- (10×5), (20×10), (30×15), (40×20), (50×25), (60×30), (70×35), (80×40), (90×45), (100×50)

## **Number of Instances per Category**  
To ensure sufficient variation across different problem sizes, I will generate 250 problems per subcategory. This is because I also want to make the running time feasible.

This results in a total of **20,000 randomly generated problems** for the experiment.

## **Metrics for Comparison**  
To evaluate the performance of different pivot rules, I will measure:

1. **Number of iterations (pivot steps) taken** to reach optimality
2. **Computation time** for each problem instance

## Imports, setup and problems creation

In [1]:

import random
import os
import sys
from dense_lp_generator import DenseLPGenerator
from simplex_solver import SimplexSolver
from input_parser import LPParser

# IMPORTANT: Set random seed for reproducibility.
random.seed(42)

balanced_sizes = [(i, i) for i in range(5, 101, 5)]
wide_sizes = [(i, 2*i) for i in range(5, 51, 5)]
tall_sizes = [(2*i, i) for i in range(5, 51, 5)]
root_folder = os.path.join('problems', 'problems_pivot_rules_one_phase_only')
balanced_folder = os.path.join(root_folder, 'balanced_problems')
wide_folder = os.path.join(root_folder, 'wide_problems')
tall_folder = os.path.join(root_folder, 'tall_problems')
random_dense_gen = DenseLPGenerator(precision = 4)

if not os.path.exists(root_folder):
    os.mkdir(root_folder)

if not os.path.exists(balanced_folder):
    os.mkdir(balanced_folder)
    
if not os.path.exists(wide_folder):
    os.mkdir(wide_folder)
    
if not os.path.exists(tall_folder):
    os.mkdir(tall_folder)

### Generating the balanced problems

In [2]:
for (x, y) in balanced_sizes:
    current_size_folder = os.path.join(balanced_folder, f'{x}x{y}')
    if not os.path.exists(current_size_folder):
        os.mkdir(current_size_folder)
    
    for i in range(250):
        random_dense_gen.generate_dense_lp(os.path.join(current_size_folder, f"{i+1}.lp"), x, y)

### Generating the wide problems

In [3]:
for (x, y) in wide_sizes:
    current_size_folder = os.path.join(wide_folder, f'{x}x{y}')
    if not os.path.exists(current_size_folder):
        os.mkdir(current_size_folder)
    
    for i in range(250):
        random_dense_gen.generate_dense_lp(os.path.join(current_size_folder, f"{i+1}.lp"), x, y)

### Generating the tall problems

In [4]:
for (x, y) in tall_sizes:
    current_size_folder = os.path.join(tall_folder, f'{x}x{y}')
    if not os.path.exists(current_size_folder):
        os.mkdir(current_size_folder)
    
    for i in range(250):
        random_dense_gen.generate_dense_lp(os.path.join(current_size_folder, f"{i+1}.lp"), x, y)

## The first experiment - Running time of Dantzig
This experiment aims to plot the average time of Dantzig's pivot rule for all problem sizes, in order to establish a feasible upper bound for our problem sizes, i.e. to be able to run the experiment in a timely manner

In [2]:
import json
import time
from tqdm import tqdm
dantzig_solver = SimplexSolver(pivot_rule = 'Dantzig')
lp_parser = LPParser()

### Running Dantzig on balanced problems

In [ ]:
for (x, y) in balanced_sizes[]:
    print(f'Currently solving for size {x}x{y}.')
    current_size_results = {}

    for i in tqdm(range(1, 251)):
        lp_parser.parse_file(os.path.join(balanced_folder, f"{x}x{y}", f"{i}.lp"))

        start_time = time.time()
        my_solver_output = dantzig_solver.solve(lp_parser)
        end_time = time.time()

        my_solver_output['total_time'] = (end_time - start_time) * 1000
        current_size_results[f"{i}.lp"] = my_solver_output
    
    with open(os.path.join(balanced_folder, f"{x}x{y}", f'dantzig_results.json'), 'w') as f:
        json.dump(current_size_results, f, indent = 4)
    
    print(f'Done with size {x}x{y}. Results saved.')

Currently solving for size 100x100.


  0%|          | 1/250 [00:18<1:15:39, 18.23s/it]


KeyboardInterrupt: 